# Artificial neural networks (ANN) coding
## For predicting the pressure loss coefficient (K) of 90degree bends 

#### This code is based on the Python 3.6.8 version and Tensorflow 1.12.0 version


## Command-line interface (CLI) 

#### By using 'argparse module', we define variables that are used to predict the K(pressure loss coefficient) value 

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
import numpy as np
import math
import argparse
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

parser=argparse.ArgumentParser(description="Prediction of the pressure loss coefficient of 90degree bends")
parser.add_argument('-d','--diameter',metavar='', required=True, type=float, help='Diameter of pipe [m]')
parser.add_argument('-R','--radius',metavar='', required=True, type=float, help='Radius of curvature [m]')
parser.add_argument('-rho','--density',metavar='', required=True, type=float, help='Density [kg/m3]')
parser.add_argument('-mu','--viscosity',metavar='', required=True, type=float, help='Dynamic  Viscosity [kg/ms]')
parser.add_argument('-Q','--flowrate', metavar='', required=True, type=float, help='Volume flow rate [m3/s]')

args=parser.parse_args()

## ANN model structure

#### This ANN model consists of 20 neurons in a hidden layer and use sigmoid function

In [ ]:
X=tf.placeholder(tf.float32 , shape=[None,2])
Y=tf.placeholder(tf.float32 , shape=[None,1])

w1=tf.get_variable("w1", shape=[2,20],initializer=tf.contrib.layers.variance_scaling_initializer())
b1=tf.Variable(tf.random_uniform([1], minval=0., maxval=1), name='bias1')
layer1=tf.sigmoid(tf.matmul(X,w1)+b1)


w2=tf.get_variable("w2", shape=[20,1],initializer=tf.contrib.layers.variance_scaling_initializer())
b2=tf.Variable(tf.random_uniform([1], minval=0., maxval=1), name='bias2')
hypo=tf.matmul(layer1,w2)+b2

cost=tf.reduce_mean(tf.square(hypo-Y))

saver = tf.train.Saver()

## Processing input data

#### The entered information is processed as input variables of the ANN model

In [ ]:
velocity = (4*args.flowrate)/(args.diameter*args.diameter*math.pi)

Velocity = round(velocity,2)

Re = (args.density*Velocity*args.diameter)/args.viscosity

PIRNT_Re=round(Re,2)

Re_log = math.log10(Re)

Rr = args.radius/(args.diameter/2)

PIRNT_Rr = round(Rr,2)

## Trained ANN model load & prediction

#### The ANN model predicts the pressure loss coefficient by using input variables
#### In addition, the pressure drop of a 90degree bend is calculated by using the K value

In [ ]:
save_file = './K_90bend.ckpt'
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, save_file)

    feed_input = np.array([[Re_log,Rr]])

    prediction = sess.run(hypo, feed_dict={X : feed_input})

    prediction_result = prediction[0,0]
    
    pressure_drop = prediction_result*args.density*0.5*Velocity*Velocity

    Pressure_drop = round(pressure_drop,2)
   
    print("---Prediction of pressure loss coefficient---\n")
    print("Prsssure loss coefficient = %f \n\n" %prediction_result)
    print("---------Calculation of pressure drop--------\n\n")
    print("Reynolds number:%f \n" %PIRNT_Re)
    print("Velocity (m/s):%f \n" %Velocity)
    print("Pressure drop (Pa): %f" %Pressure_drop)

#### This ANN model can predict valid results in the range below

#### - 10,000 < Reynolds number (Re) < 600,000
#### - 1.1 < R/r < 58.4

#### Copyright 2021.(AFNTL at Chonnam National University) all rights reserved.